<a href="https://colab.research.google.com/github/SatyamRaj1/Bias-Detection/blob/main/SVM_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
from tqdm import tqdm

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Data/ToxicBias")

#Import data

In [ ]:
# # Initialize an empty dictionary to store the data frames
dfs = {}
# # Load each CSV file into a Pandas DataFrame and store it in the dictionary with appropriate keys
# # Now, you have three data frames: data_frames['train'], data_frames['test'], and data_frames['validation']
# # You can access and manipulate these data frames as needed.
dfs['train'] = pd.read_csv('toxicbias_train.csv')
dfs['test'] = pd.read_csv('toxicbias_test.csv')
dfs['validation'] = pd.read_csv('toxicbias_val.csv')

In [ ]:
dfs['train'].columns

Index(['comment_text', 'identity_attack_x', 'bias', 'target', 'category',
       'rationale'],
      dtype='object')

In [ ]:
# Import necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have three dataframes: dfs['train'], dfs['test'], and dfs['validation']

# Data Preprocessing
# Assuming 'bias' is binary (biased or not biased)
# Convert 'bias' to 0 (not biased) and 1 (biased)
dfs['train']['bias'] = dfs['train']['bias'].apply(lambda x: 1 if x == 'bias' else 0)
dfs['test']['bias'] = dfs['test']['bias'].apply(lambda x: 1 if x == 'bias' else 0)
dfs['validation']['bias'] = dfs['validation']['bias'].apply(lambda x: 1 if x == 'bias' else 0)

In [ ]:
count_of_neutral = dfs['train']['bias'].value_counts()[0]
count_of_neutral

867

# Logit

In [ ]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')  # You can adjust max_features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(dfs['train']['comment_text'])
X_test_tfidf = tfidf_vectorizer.transform(dfs['test']['comment_text'])
X_validation_tfidf = tfidf_vectorizer.transform(dfs['validation']['comment_text'])

# Create and Train the Logistic Regression Model
lr_model = LogisticRegression()
lr_model.fit(X_train_tfidf, dfs['train']['bias'])

# Make predictions on the test set
y_pred = lr_model.predict(X_test_tfidf)

# Evaluate the model on the test set
accuracy = accuracy_score(dfs['test']['bias'], y_pred)
print("Test Set Accuracy:", accuracy)

# You can also print a classification report for more detailed metrics
print("Test Set Classification Report:")
print(classification_report(dfs['test']['bias'], y_pred))

# Now you can use this model to make predictions on the validation set
y_val_pred = lr_model.predict(X_validation_tfidf)

# Evaluate the model on the validation set
val_accuracy = accuracy_score(dfs['validation']['bias'], y_val_pred)
print("Validation Set Accuracy:", val_accuracy)

# To predict the bias of a new comment
new_comment = "This is a biased comment"
new_comment_tfidf = tfidf_vectorizer.transform([new_comment])
prediction = lr_model.predict(new_comment_tfidf)
if prediction[0] == 1:
    print("The comment is biased.")
else:
    print("The comment is not biased.")

Test Set Accuracy: 0.8
Test Set Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.01      0.02       131
           1       0.80      1.00      0.89       519

    accuracy                           0.80       650
   macro avg       0.90      0.50      0.45       650
weighted avg       0.84      0.80      0.71       650

Validation Set Accuracy: 0.8009259259259259
The comment is not biased.


In [ ]:
# To predict the bias of a new comment
new_comment = "This is not a good model"
new_comment_tfidf = tfidf_vectorizer.transform([new_comment])
prediction = lr_model.predict(new_comment_tfidf)
if prediction[0] == 1:
    print("The comment is biased.")
else:
    print("The comment is not biased.")

The comment is biased.


# SVM

In [ ]:
# Import necessary libraries
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have three dataframes: dfs['train'], dfs['test'], and dfs['validation']

# Create and Train the SVM Model with Linear Kernel
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_tfidf, dfs['train']['bias'])

# Make predictions on the test set
y_pred = svm_model.predict(X_test_tfidf)

# Evaluate the model on the test set
accuracy = accuracy_score(dfs['test']['bias'], y_pred)
print("Test Set Accuracy:", accuracy)

# You can also print a classification report for more detailed metrics
print("Test Set Classification Report:")
print(classification_report(dfs['test']['bias'], y_pred))

# Now you can use this model to make predictions on the validation set
y_val_pred = svm_model.predict(X_validation_tfidf)

# Evaluate the model on the validation set
val_accuracy = accuracy_score(dfs['validation']['bias'], y_val_pred)
print("Validation Set Accuracy:", val_accuracy)

# To predict the bias of a new comment
new_comment = "This is a biased comment"
new_comment_tfidf = tfidf_vectorizer.transform([new_comment])
prediction = svm_model.predict(new_comment_tfidf)
if prediction[0] == 1:
    print("The comment is biased.")
else:
    print("The comment is not biased.")


Test Set Accuracy: 0.8015384615384615
Test Set Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.02      0.04       131
           1       0.80      1.00      0.89       519

    accuracy                           0.80       650
   macro avg       0.78      0.51      0.47       650
weighted avg       0.79      0.80      0.72       650

Validation Set Accuracy: 0.7962962962962963
The comment is not biased.


# LSTM

In [ ]:
import torch
import torch.nn as nn
!pip install torchtext==0.6 torch==1.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement torch==1.5 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0)
ERROR: No matching distribution found for torch==1.5


In [ ]:
from torchtext.legacy.data import Field, TabularDataset, BucketIterator

ModuleNotFoundError: ignored

In [ ]:
# Assuming you have dfs['train'], dfs['test'], and dfs['validation'] dataframes

# Define the Fields for text and labels
TEXT = Field(sequential=True, tokenize='spacy', lower=True, include_lengths=True)
LABEL = Field(sequential=False, use_vocab=False)

# Create TabularDatasets
fields = [('comment_text', TEXT), ('bias', LABEL)]
train_data, test_data, val_data = TabularDataset.splits(
    path='.', train=dfs['train'], validation=dfs['validation'], test=dfs['test'], format='csv', fields=fields)

# Build the vocabulary using pre-trained GloVe word vectors
TEXT.build_vocab(train_data, vectors="glove.6B.300d")
# If you don't have GloVe vectors, you can train them from scratch or download them

# Create Iterators to batch the data
BATCH_SIZE = 64
train_iterator, test_iterator, val_iterator = BucketIterator.splits(
    (train_data, test_data, val_data), batch_size=BATCH_SIZE, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super(LSTMModel, self).__init()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1) if self.lstm.bidirectional else hidden[-1, :, :])
        return self.fc(hidden)

# Define hyperparameters
VOCAB_SIZE = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 1  # Binary classification
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

# Create the model
model = LSTMModel(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

# Use pre-trained word embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

# Place the model and loss function on the GPU (if available)
model = model.to('cuda')
criterion = criterion.to('cuda')

# Training function
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for batch in iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.comment_text
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.bias.float())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# Evaluation function
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.comment_text
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.bias.float())
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# Training loop
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    train_loss = train(model, train_iterator, optimizer, criterion)
    valid_loss = evaluate(model, val_iterator, criterion)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'lstm-model.pt')  # Save the best model

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tVal Loss: {valid_loss:.3f}')

# Load the best model and test it
model.load_state_dict(torch.load('lstm-model.pt'))
test_loss = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f}')

# Make predictions for a new comment
def predict_sentiment(model, tokenizer, comment):
    model.eval()
    tokenized = tokenizer(comment)
    indexed = [TEXT.vocab.stoi[x] for x in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to('cuda')
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

new_comment = "This is a biased comment"
predicted_bias = predict_sentiment(model, TEXT, new_comment)
if predicted_bias > 0.5:
    print("The comment is biased.")
else:
    print("The comment is not biased.")


# BERT Heirachal

In [ ]:
!pip install transformers

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Preprocess the text data and labels
# You'll need to tokenize the text and convert labels to one-hot encoding.

# 2. Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load your data
train_df = dfs['train']
test_df = dfs['test']
validation_df = dfs['validation']

# Preprocess the text and labels
def preprocess_data(df, max_length):
    tokenized_text = []
    for text in df['comment_text']:
        encoded_text = tokenizer.encode(text, add_special_tokens=True, max_length=max_length, truncation=True)
        tokenized_text.append(encoded_text)

    input_ids = torch.tensor([text + [tokenizer.pad_token_id] * (max_length - len(text)) for text in tokenized_text]).to(device)

    label_mapping = {'neutral': 0, 'bias': 1}
    labels = torch.tensor([label_mapping[label] for label in df['bias']]).to(device)
    return input_ids, labels

# Set the maximum sequence length
max_length = 128

# Preprocess the data for training, validation, and test
train_input_ids, train_labels = preprocess_data(train_df, max_length)
validation_input_ids, validation_labels = preprocess_data(validation_df, max_length)
test_input_ids, test_labels = preprocess_data(test_df, max_length)

# Create data loaders
batch_size = 32

train_data = DataLoader(TensorDataset(train_input_ids, train_labels), batch_size=batch_size, shuffle=True)
validation_data = DataLoader(TensorDataset(validation_input_ids, validation_labels), batch_size=batch_size)
test_data = DataLoader(TensorDataset(test_input_ids, test_labels), batch_size=batch_size)

# 4. Define the bias detection and category classification models
# You need to create a separate model for category classification.

class CategoryClassificationModel(nn.Module):
    def __init__(self, num_categories):
        super(CategoryClassificationModel, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_categories).to(device)

In [ ]:
# 5. Define loss functions and optimizers
bias_detection_criterion = nn.CrossEntropyLoss()
category_classification_criterion = nn.BCEWithLogitsLoss()
bias_detection_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)
optimizer = AdamW(bias_detection_model.parameters(), lr=0.001, eps=1e-8)

# 6. Train the bias detection model
num_epochs = 100
for epoch in range(num_epochs):
    for batch in train_data:
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = bias_detection_model(inputs)
        loss = bias_detection_criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

# 7. Filter the input data using the trained bias detection model
filtered_data = []
for text, label in test_data:
    with torch.no_grad():
        outputs = bias_detection_model(text)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        filtered_data.append((text[predicted_labels == 1], label[predicted_labels == 1]))

# 8. Create data loaders for category classification
category_train_data = DataLoader(category_train_inputs, category_train_labels, batch_size=batch_size, shuffle=True)
category_validation_data = DataLoader(category_validation_inputs, category_validation_labels, batch_size=batch_size)

# 9. Train the category classification model
category_model = CategoryClassificationModel(num_categories).to(device)
optimizer_category = AdamW(category_model.parameters(), lr=..., eps=...)

for epoch in range(num_epochs):
    for batch in category_train_data:
        inputs, labels = batch
        optimizer_category.zero_grad()
        outputs = category_model(inputs)
        loss = category_classification_criterion(outputs.logits, labels)
        loss.backward()
        optimizer_category.step()

# Evaluate the hierarchical model on the test dataset
category_model.eval()
with torch.no_grad():
    for batch in test_data:
        inputs, labels = batch
        outputs = category_model(inputs)
        # Evaluate the category classification results


# BERT Multitask

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Load your data
train_df = dfs['train']
test_df = dfs['test']
validation_df = dfs['validation']

# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

In [ ]:
# Define your device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the batch size and maximum sequence length
batch_size = 16
max_seq_length = 128

# Define a custom dataset for bias detection and bias category classification
class CustomMultiTaskDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Define a mapping of labels to numerical values for bias detection
        self.bias_label_map = {"bias": 1, "neutral": 0}

        # Define a mapping of labels to numerical values for bias categories
        self.category_label_map = {"Political": 0, "Religion": 1, "Gender": 2, "LGBTQ": 3, "Race": 4}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Extract text, bias, and category labels from the DataFrame
        text = self.data['comment_text'].iloc[idx]
        bias_labels = self.bias_label_map[self.data['bias'].iloc[idx]]  # Map bias labels to numerical values

        # Map category labels to numerical values
        # If the category is not recognized, assign -1 (you can handle this case as needed)
        category_labels = self.category_label_map.get(self.data['category'].iloc[idx], -1)

        if category_labels == -1:
            # Handle unknown categories or 'None' values
            # You can choose how to handle these cases, e.g., assign them to a default category or ignore them.
            pass

        # Tokenize the text and create input tensors
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length,
                                  return_tensors='pt', return_attention_mask=True)
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,  # Input IDs for BERT
            'attention_mask': attention_mask,  # Attention mask for BERT
            'bias_labels': torch.tensor(bias_labels, dtype=torch.long),  # Numerical bias label
            'category_labels': torch.tensor(category_labels, dtype=torch.long)  # Numerical category label
        }

In [ ]:
# Create data loaders for bias detection and bias category classification
train_dataset = CustomMultiTaskDataset(train_df, tokenizer, max_seq_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = CustomMultiTaskDataset(validation_df, tokenizer, max_seq_length)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size)

test_dataset = CustomMultiTaskDataset(test_df, tokenizer, max_seq_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Define a multi-task model with two output heads
class MultiTaskBERT(torch.nn.Module):
    def __init__(self, pretrained_model_name, num_categories):
        super(MultiTaskBERT, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=2)  # Bias detection head
        self.category_head = torch.nn.Linear(self.bert.config.hidden_size, num_categories)  # Bias category head

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        logits = outputs[0]  # Logits for bias detection

        # Extract the [CLS] token embedding for category classification
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # [CLS] token embedding
        bias_logits = logits
        category_logits = self.category_head(cls_embedding)  # Logits for category classification
        return bias_logits, category_logits

# Create the multi-task model
model = MultiTaskBERT("bert-base-uncased", num_categories=5).to(device)

In [ ]:
model

In [ ]:
def train(model, train_loader, validation_loader, epochs, device):
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in tqdm(range(epochs)):
        model.train()
        total_loss = 0.0

        # Training loop
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            bias_labels = batch['bias_labels'].to(device)  # Labels for bias detection
            category_labels = batch['category_labels'].to(device)  # Labels for bias category classification

            optimizer.zero_grad()

            # Forward pass through the model
            bias_logits, category_logits = model(input_ids, attention_mask=attention_mask)

            # Calculate loss for bias detection
            bias_loss = criterion(bias_logits, bias_labels)

            # Calculate loss for bias category classification
            # Ensure you handle unknown categories appropriately in your data preprocessing
            # Here, we ignore unknown categories (-1)
            known_category_mask = category_labels != -1
            category_loss = criterion(category_logits[known_category_mask], category_labels[known_category_mask])

            # Total loss is a combination of bias and category losses
            total_loss = bias_loss + category_loss
            total_loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

        model.eval()
        val_loss = 0.0
        bias_predictions = []
        category_predictions = []
        true_bias_labels = []
        true_category_labels = []

        # Validation loop
        with torch.no_grad():
            for batch in validation_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                bias_labels = batch['bias_labels'].to(device)
                category_labels = batch['category_labels'].to(device)
                bias_logits, category_logits = model(input_ids, attention_mask=attention_mask)

                # Calculate validation loss for bias detection
                val_loss += criterion(bias_logits, bias_labels)

                # Store predictions and true labels
                bias_predictions.extend(bias_logits.argmax(dim=1).tolist())
                category_predictions.extend(category_logits.argmax(dim=1).tolist())
                true_bias_labels.extend(bias_labels.tolist())
                true_category_labels.extend(category_labels.tolist())

        print(f"Validation Loss: {val_loss / len(validation_loader)}")

        # Calculate accuracy for bias detection
        bias_acc = accuracy_score(true_bias_labels, bias_predictions)

        # Calculate accuracy for bias category classification
        known_category_mask = [category != -1 for category in true_category_labels]
        category_acc = accuracy_score([true_category_labels[i] for i, known in enumerate(known_category_mask) if known],
                                      [category_predictions[i] for i, known in enumerate(known_category_mask) if known])

        # Print the validation accuracy for both bias detection and category classification
        print(f"Validation Bias Accuracy: {bias_acc}")
        print(f"Validation Category Accuracy: {category_acc}")
        # Additional classification reports or metrics can be added as needed



In [ ]:
# Train the model
train(model, train_loader, validation_loader, epochs=3, device = device)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def test(model, test_loader, device):
    model.eval()
    test_loss = 0.0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            test_loss += outputs.loss.item()

            logits = outputs.logits
            predicted_labels = logits.argmax(dim=1)

            predictions.extend(predicted_labels.tolist())
            true_labels.extend(labels.tolist())

    test_loss /= len(test_loader)
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    print(f"Test Loss: {test_loss}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision (Macro): {precision}")
    print(f"Recall (Macro): {recall}")
    print(f"F1 Score (Macro): {f1}")

In [ ]:
# To test the model on the test data
test(model, test_loader, device)

#BERT MUlti again

In [ ]:
!pip install transformers
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Load your data
train_df = dfs['train']
test_df = dfs['test']
val_df = dfs['validation']

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length, num_categories):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.num_categories = num_categories
        self.category_list = ['political', 'religion', 'gender', 'lgbtq', 'race']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        comment_text = self.data['comment_text'].iloc[index]
        bias_label = 1 if self.data['bias'].iloc[index] == 'bias' else 0
        category_label = [0] * self.num_categories

        # Assuming the 'category' column is a comma-separated string of categories
        categories = self.data['category'].iloc[index].split(', ')

        # Set category_label to 1 for each category present in the comment
        for category in categories:
            if category in ['political', 'religion', 'gender', 'lgbtq', 'race']:
                category_label[self.category_list.index(category)] = 1

        encoding = self.tokenizer.encode_plus(
            comment_text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': torch.tensor(encoding['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(encoding['attention_mask'], dtype=torch.long),
            'bias_label': torch.tensor(bias_label, dtype=torch.float),
            'category_labels': torch.tensor(category_label, dtype=torch.float)
        }

# Set the maximum sequence length for padding/truncation
max_length = 128
num_categories = 5  # Number of categories: Political, Religion, Gender, LGBTQ, Race

# Assuming you have 'train_df', 'val_df', and 'test_df'
train_dataset = CustomDataset(train_df, tokenizer, max_length, num_categories)
val_dataset = CustomDataset(val_df, tokenizer, max_length, num_categories)
test_dataset = CustomDataset(test_df, tokenizer, max_length, num_categories)

# Create data loaders for each dataset
batch_size = 16  # You can adjust this based on your memory capacity
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

# Initialize BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Define your custom MTL model
class MultiTaskBERT(nn.Module):
    def __init__(self, num_categories):
        super(MultiTaskBERT, self).__init__()
        self.bert = bert_model
        self.bias_classifier = nn.Linear(768, 1)  # One neuron for bias detection
        self.category_classifier = nn.Linear(768, num_categories)  # Five neurons for category classification

    def forward(self, input_ids, attention_mask):
        # BERT forward pass
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs['pooler_output']

        # Bias classification
        bias_output = self.bias_classifier(pooled_output)

        # Category classification
        category_output = self.category_classifier(pooled_output)

        return bias_output, category_output

# Create an instance of the MTL model
num_categories = 5  # Political, Religion, Gender, LGBTQ, Race
model = MultiTaskBERT(num_categories)


In [ ]:
bias_criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss for bias detection
category_criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss for category classification
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for batch in train_data_loader:
        input_ids, attention_mask, bias_labels, category_labels = batch

        optimizer.zero_grad()

        bias_output, category_output = model(input_ids, attention_mask)

        bias_loss = bias_criterion(bias_output, bias_labels)
        category_loss = category_criterion(category_output, category_labels)

        total_loss = bias_loss + category_loss
        total_loss.backward()
        optimizer.step()

#     # Evaluate the model on the validation data
#     model.eval()
#     with torch.no_grad():
#         # Calculate validation loss and accuracy

# # Test the model on the test data
# model.eval()
# with torch.no_grad():
#     # Calculate test loss and accuracy
